# Data Read & Exploration

In [1]:
import pandas as pd
df = pd.read_csv("./cafe_sales.csv")
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [3]:
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

# Data Cleaning

**Dropped 'Transaction ID' column**

In [4]:
df2 = df.drop('Transaction ID', axis=1)
df2.head(5)

,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


**Data types**

In [5]:
# object to categorical
df2['Item'] = df2['Item'].astype('category')
df2['Payment Method'] = df2['Payment Method'].astype('category')
df2['Location'] = df2['Location'].astype('category')

# object to numerical
df2['Quantity'] = pd.to_numeric(df2['Quantity'], errors='coerce')
df2['Price Per Unit'] = pd.to_numeric(df2['Price Per Unit'], errors='coerce')
df2['Total Spent'] = pd.to_numeric(df2['Total Spent'], errors='coerce')

# object to date
df2['Transaction Date'] = pd.to_datetime(df2['Transaction Date'], format='mixed', errors='coerce').dt.date

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Item              9667 non-null   category
 1   Quantity          9521 non-null   float64 
 2   Price Per Unit    9467 non-null   float64 
 3   Total Spent       9498 non-null   float64 
 4   Payment Method    7421 non-null   category
 5   Location          6735 non-null   category
 6   Transaction Date  9540 non-null   object  
dtypes: category(3), float64(3), object(1)
memory usage: 342.7+ KB


**Replace and fill null**

In [6]:
#replace "ERROR" with "UNKNOWN"
df2 = df2.replace( 'ERROR', 'UNKNOWN')

#replace "UNKNOWN" with NaN
columns = ['Item', 'Price Per Unit', 'Total Spent', 'Quantity']
df2[columns] = df2[columns].replace('UNKNOWN', pd.NA)

# fill null values with "UNKNOWN"
df2['Payment Method'] = df2['Payment Method'].fillna("UNKNOWN")
df2['Location'] = df2['Location'].fillna("UNKNOWN")
df2['Transaction Date'] = df2['Transaction Date'].fillna("UNKNOWN")

df2.isnull().sum()

C:\Users\ashiq\AppData\Local\Temp\ipykernel_15324\1174652641.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df2 = df2.replace( 'ERROR', 'UNKNOWN')
C:\Users\ashiq\AppData\Local\Temp\ipykernel_15324\1174652641.py:6: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df2[columns] = df2[columns].replace('UNKNOWN', pd.NA)


Item                969
Quantity            479
Price Per Unit      533
Total Spent         502
Payment Method        0
Location              0
Transaction Date      0
dtype: int64

**Menu items with price**

In [7]:
menu_price = {
    'Coffee': 2.0,
    'Tea': 1.5,
    'Sandwich': 4.0,
    'Salad': 5.0,
    'Cake': 3.0,
    'Cookie': 1.0,
    'Smoothie': 4.0,
    'Juice': 3.0
}

price_to_item = {v: k for k, v in menu_price.items()} # price_to_item = {2.0 : "Coffee", ...}

**Fill numerical values**

In [8]:
# filling "Items" by extracting menu_price with "Price Per Unit"
df2['Item'] = df2['Item'].fillna(df2['Price Per Unit'].map(price_to_item))

# filling "Price Per Unit" by extracting price_to_item with"Items"
df2['Price Per Unit'] = df2['Price Per Unit'].fillna(df2['Item'].map(menu_price))

# Price per uni = total spent / quantity
df2['Price Per Unit'] = df2['Price Per Unit'].fillna(df2['Total Spent'] // df2['Quantity'])

# again filling Items
df2['Item'] = df2['Item'].fillna(df2['Price Per Unit'].map(price_to_item))

# quantity = total spent / price per unit
df2['Quantity'] = df2['Quantity'].fillna(df2['Total Spent'] // df2['Price Per Unit'])

# total spent = quantity * price per unit
df2['Total Spent'] = df2['Total Spent'].fillna(df2['Quantity'] * df2['Price Per Unit'])

**Drop remaining null rows**

In [9]:
print("Before dropping null rows\n", df2.isnull().sum())
df2.dropna(inplace=True)
print("\nAfter dropping null rows\n", df2.isnull().sum())

Before dropping null rows
 Item                 6
Quantity            23
Price Per Unit       6
Total Spent         23
Payment Method       0
Location             0
Transaction Date     0
dtype: int64

After dropping null rows
 Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64


# Saving to CSV

In [10]:
df2.to_csv('processed_data.csv', index=False)